# Desafio 1

link a los datos: http://campus.digitalhouse.com/mod/url/view.php?id=32168
Es necesario guardar el csv con el nombre properatti.csv en una carpeta DATA en el raiz del proyecto

In [1]:
#Importar librerias
import numpy as np
import pandas as pd
import re
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Leemos el archivo y creamos el dataframe "data"
data = pd.read_csv("./DATA/properatti.csv")
data.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [3]:
#1. #Renombramos Columnas
data.rename(columns={'Unnamed: 0':'ID',
                    'property_type':'TIPO_PROPIEDAD'}, inplace = True) 

In [4]:
#2. Agregamos 4 columnas al dataframe "data" (PAIS/PROVINCIA/CIUDAD/BARRIO) a partir de place_with_parent_names,place_name.

#2.1 Spliteamos para sacar el pipe y creamos un nuevo dataframe (df_ubicacion) que luego joinearemos para pegarle las 4 columnas

pais_lista =  [elemento[1] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
provincia_region_lista =  [elemento[2] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
barrio_ciudad =  [elemento[3] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
barrio  = data['place_name'].tolist()

d = {'PAIS':pais_lista ,
     'PROVINCIA':  provincia_region_lista  ,
     'CIUDAD':barrio_ciudad,
     'BARRIO':barrio,
     'ID': data['ID'].tolist()
    }
df_ubicacion = pd.DataFrame(data=d)

#Si PROVINCIA = CABA -> a ciudad asignar CABA.
df_ubicacion['CIUDAD'] = np.where(df_ubicacion['PROVINCIA'] =='Capital Federal','Capital Federal',df_ubicacion['CIUDAD'])

#Si CIUDAD = BARRIO -> BARRIO DEJARLO EN NAN
df_ubicacion['BARRIO'] = np.where(df_ubicacion['BARRIO'] ==df_ubicacion['CIUDAD'],'SIN DATO',df_ubicacion['BARRIO'])

#Si CIUDAD esta vacio ('') y barrio == a pronvincia , llenar CIUDAD Y BARRIO CON NAN
df_ubicacion['BARRIO'] = np.where((df_ubicacion['BARRIO'] ==df_ubicacion['PROVINCIA']) & (df_ubicacion['CIUDAD'] =='') ,'SIN DATO',df_ubicacion['BARRIO'])
df_ubicacion['CIUDAD'] = np.where((df_ubicacion['BARRIO'] ==df_ubicacion['PROVINCIA']) & (df_ubicacion['CIUDAD'] =='') ,'SIN DATO',df_ubicacion['CIUDAD'])
df_ubicacion['CIUDAD'] = np.where(df_ubicacion['CIUDAD'] =='', 'SIN DATO',df_ubicacion['CIUDAD'])

#2.2agregamos esta informacion al data frame DATA
data = pd.merge(data, df_ubicacion, on='ID', how='left')
data.rename(columns={'CIUDAD':'CIUDAD-PARTIDO'}, inplace=True)



In [5]:
patron = r'(\d*|mono|\w*)\s*?amb\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["title_dato"] = data.title.str.extract(r'(\d*|mono|\w*)\s*?amb\w*', expand=False)

In [6]:
patron = r'(\d*|MONO|\w*)\s*?AMB\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["title_dato1"] = data.title.str.extract(r'(\d*|MONO|\w*)\s*?AMB\w*', expand=False)

In [7]:
patron = r'(\d*|mono|\w*)\s*?amb\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["description_dato"] = data.description.str.extract(r'(\d*|mono|\w*)\s*?amb\w*', expand=False)

In [8]:
patron = r'(\d*|MONO|\w*)\s*?AMB\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["description_dato1"] = data.description.str.extract(r'(\d*|MONO|\w*)\s*?AMB\w*', expand=False)

In [9]:
# Reemplazar los valores NaN por 0

In [10]:
data["title_dato"]= data["title_dato"].fillna(0)
data["title_dato1"]= data["title_dato1"].fillna(0)
data["description_dato"]= data["description_dato"].fillna(0)
data["description_dato1"]= data["description_dato1"].fillna(0)

In [11]:
# Generar datos de ambientes con palabras clave

In [12]:
data.replace({'title_dato': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO":1, "mono":1, "DOS":2, "dos":2, "TRES":3, "tres":3, "CUATRO":4, "cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'title_dato1': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'description_dato': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'description_dato1': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)

In [13]:
# Los valores texto indetrminados los convierto en 0

In [14]:
data["title_dato"]=data["title_dato"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["title_dato1"]=data["title_dato1"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["description_dato"]=data["description_dato"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["description_dato1"]=data["description_dato1"].replace(r'[a-z]|[A-Z]', 0, regex=True)

In [15]:
#convierto los datos a numeric para poder calcular los ambientes

In [16]:
data[['title_dato','title_dato1','description_dato','description_dato1']] = data[['title_dato','title_dato1','description_dato','description_dato1']].apply(pd.to_numeric)

In [17]:
#Calculo los ambientes

In [18]:
data['Ambientes']=data[['title_dato','title_dato1','description_dato','description_dato1']].apply(np.max,axis=1)

In [19]:
#Convierto Ambientes a int 

In [20]:
data['Ambientes']=data['Ambientes'].astype(int)

In [21]:
#elimino las columnas de calculo

In [22]:
data = data.drop('title_dato', 1)
data = data.drop('title_dato1', 1)
data = data.drop('description_dato', 1)
data = data.drop('description_dato1', 1)

In [23]:
#FIN AMBIENTES

In [24]:
# Detectar dptos a pozo para eliminar del análisis - Juan
patron = r'(pozo|cuota|proyec|emprendim)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["pozo"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

# Y con esto los elimino
data = data[data.pozo == 0]
data = data.drop(['pozo'], axis = 1)

In [25]:
# Agregar dummies de características que puedan influenciar (ammenities, cochera, etc.) - Juan
patron = r'(pileta|pisci)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["pileta"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(ameni)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["amenities"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(gimn|gym)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["gimnasio"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(laundry)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["laundry"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(lavader)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["lavadero"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(balcón|balcon)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["balcon"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(terraz)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["terraza"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(sum|usos mult)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["sum"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(solarium)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["solarium"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(parr)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["parrilla"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(estre)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["a estrenar"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(subte)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["subte"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(garage|cocher)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["cochera"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

# Contar casos de cada una de las características

data.loc[:,'pileta':'cochera'].sum()

pileta        25811
amenities      6927
gimnasio       6474
laundry        4305
lavadero      35652
balcon        34162
terraza       21627
sum           10175
solarium       6137
parrilla      29718
a estrenar     9323
subte          5640
cochera       40946
dtype: int64

In [26]:
# Borramos columnas que ya no necesitamos 
del data['place_name']
del data['place_with_parent_names']
del data['country_name']
del data['state_name']
del data['geonames_id']
del data['operation']


data.head()

,ID,TIPO_PROPIEDAD,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,...,laundry,lavadero,balcon,terraza,sum,solarium,parrilla,a estrenar,subte,cochera
0,0,PH,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,...,0,1,0,0,0,0,0,0,0,0
1,1,apartment,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,...,0,1,1,0,0,0,0,1,0,1
2,2,apartment,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,...,0,1,0,0,0,0,0,0,0,0
3,3,PH,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,apartment,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#pd.set_option('display.height', 1000)
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

data['surface_total_in_m2'] = np.where(data['surface_total_in_m2'].isnull() | data['surface_total_in_m2'] == 0 
                                       ,data['surface_covered_in_m2'],data['surface_total_in_m2'])
data['surface_covered_in_m2'] = np.where(data['surface_covered_in_m2'].isnull() | data['surface_covered_in_m2'] == 0 
                                         ,data['surface_total_in_m2'],data['surface_covered_in_m2'])
data['price_usd_per_m2'] = np.where((data['price_usd_per_m2'].isnull() | data['price_usd_per_m2'] == 0) 
                                    & (data['surface_total_in_m2'] > 0 & data['surface_total_in_m2'].notnull())
                                    ,data['price_aprox_usd'] / data['surface_total_in_m2'], data['price_usd_per_m2'])
data_final = data.dropna(subset=['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2'])
data_final.describe()

,ID,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,Ambientes,pileta,amenities,gimnasio,laundry,lavadero,balcon,terraza,sum,solarium,parrilla,a estrenar,subte,cochera
count,52624.000000,31999.000000,31999.000000,5.262400e+04,5.262400e+04,5.262400e+04,52624.000000,52624.000000,52624.000000,52623.000000,2419.000000,19713.000000,9.541000e+03,52624.00000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000,52624.000000
mean,58021.049920,-34.844925,-58.792978,4.051149e+05,4.560576e+06,2.584701e+05,234.618463,134.964674,1929.092466,4924.983221,4.095081,2.995840,5.685236e+03,3.96380,0.271872,0.079108,0.071032,0.053170,0.372321,0.353318,0.231947,0.110672,0.071811,0.333688,0.103014,0.069702,0.407362
std,32978.147384,1.637056,1.668670,8.596996e+05,6.127649e+06,3.472838e+05,1546.923896,431.453263,1563.252330,12596.531370,4.864420,1.440966,1.460137e+05,233.76781,0.444929,0.269910,0.256881,0.224374,0.483428,0.478005,0.422080,0.313728,0.258178,0.471534,0.303980,0.254647,0.491348
min,0.000000,-54.806598,-71.663515,7.500000e+03,8.906114e+04,5.047530e+03,10.000000,1.000000,0.600000,1.510204,1.000000,1.000000,1.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30259.750000,-34.650491,-58.594465,1.130000e+05,1.729161e+06,9.800000e+04,53.000000,47.000000,1144.129582,1535.043860,2.000000,2.000000,1.058000e+03,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,56191.500000,-34.599293,-58.456686,1.900000e+05,2.823120e+06,1.600000e+05,95.000000,80.000000,1754.385965,2155.555556,3.000000,3.000000,2.000000e+03,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,85431.500000,-34.541946,-58.391689,3.800000e+05,5.240416e+06,2.970000e+05,225.000000,162.000000,2432.432432,3108.108108,6.000000,4.000000,4.000000e+03,3.00000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,121219.000000,-24.166966,-54.556127,5.000000e+07,2.117340e+08,1.200000e+07,200000.000000,49000.000000,80000.000000,650000.000000,133.000000,22.000000,1.000150e+07,28612.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [62]:
pd.set_option('display.max_columns', 500)

In [63]:
# Limpio valores extremos
data_final = data_final[(data_final.price_usd_per_m2 < 5500) & (data_final.price_usd_per_m2 > 1000)]
data_final = data_final[(data_final.surface_total_in_m2 < 1000) & (data_final.surface_total_in_m2 > 20)]

In [64]:
# Algún tratamiento de outlyers?

#Devuelve como queda distribuida la base mirando PRECIO EN DOLARES
#data.price_aprox_usd.describe().astype(int)
#Devuelve como queda distribuida la base mirando PRECIO EN DOLARES tomando 2 desvidos standard
#data[np.abs(data.price_aprox_usd-data.price_aprox_usd.mean()) <= (2*data.price_aprox_usd.std())].price_aprox_usd.describe().astype(int)
#Filtrar a aquellos que quedan mas alla de 2 desvios standard. OJO QUE AL EJECUTAR PISA EL DATAFRAME
#data = data[np.abs(data.price_aprox_usd-data.price_aprox_usd.mean()) <= (3*data.price_aprox_usd.std())]
data_final.describe()

,ID,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,Ambientes,pileta,amenities,gimnasio,laundry,lavadero,balcon,terraza,sum,solarium,parrilla,a estrenar,subte,cochera
count,40675.000000,24637.000000,24637.000000,4.067500e+04,4.067500e+04,4.067500e+04,40675.000000,40675.000000,40675.000000,40675.000000,2354.000000,15511.000000,8.874000e+03,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000
mean,60739.269650,-34.839618,-58.642585,3.759226e+05,4.195848e+06,2.377992e+05,118.902200,107.104118,2138.306522,5205.258114,4.156330,2.765263,5.817462e+03,4.801967,0.260381,0.093817,0.080369,0.064831,0.351076,0.416374,0.238623,0.128728,0.084229,0.322557,0.118795,0.086392,0.405507
std,32899.987814,1.374966,1.343344,6.473110e+05,4.248888e+06,2.408052e+05,113.846588,268.364516,813.810879,11601.391773,4.911736,1.316765,1.513915e+05,265.889609,0.438848,0.291577,0.271867,0.246230,0.477312,0.492963,0.426247,0.334902,0.277734,0.467460,0.323551,0.280946,0.490996
min,0.000000,-54.806598,-71.663515,2.500000e+04,4.411125e+05,2.500000e+04,21.000000,1.000000,1000.158833,1.510204,1.000000,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33212.000000,-34.632408,-58.552965,1.100000e+05,1.711516e+06,9.700000e+04,49.000000,44.000000,1523.809524,1775.510204,2.000000,2.000000,1.100000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,61393.000000,-34.597511,-58.448754,1.800000e+05,2.770186e+06,1.570000e+05,75.000000,66.000000,1984.615385,2346.153846,3.000000,3.000000,2.000000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,88319.000000,-34.551587,-58.395752,3.600000e+05,5.099260e+06,2.890000e+05,149.000000,128.000000,2569.382792,3193.159736,6.000000,3.500000,4.000000e+03,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,121219.000000,-24.181316,-54.584900,3.000000e+07,6.352020e+07,3.600000e+06,996.000000,49000.000000,5494.505495,650000.000000,133.000000,22.000000,1.000150e+07,28612.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [65]:
data_aux = data_final[['surface_total_in_m2', 'Ambientes']]
data_aux_2 = data_aux[data_aux.Ambientes != 0]
data_aux_2.head()

,surface_total_in_m2,Ambientes
0,55.0,2
2,55.0,2
4,35.0,2
7,45.0,1
8,65.0,2


In [66]:
data_aux.groupby(by = 'Ambientes').describe()

surface_total_in_m2                                               \
                        count        mean         std    min    25%    50%   
Ambientes                                                                    
0                     18474.0  163.036267  138.441454   21.0   60.0  118.0   
1                      4194.0   43.085837   39.542878   21.0   32.0   36.0   
2                      7283.0   54.451874   31.791786   21.0   42.0   50.0   
3                      6244.0   83.021621   35.759968   25.0   63.0   75.0   
4                      3294.0  132.175167   67.698650   25.0   89.0  115.0   
5                       834.0  226.208633   99.747711   45.0  160.0  205.0   
6                       212.0  284.400943  115.619430   74.0  201.5  274.5   
7                        95.0  351.968421  173.706003  110.0  214.5  314.0   
8                        20.0  323.350000  228.900179  116.0  138.5  257.0   
9                         7.0  233.857143   95.065641  132.0  180.0  180.0   
10                        1.0  840.000000         NaN  840.0  840.0  840.0   
11                        2.0  275.000000   35.355339  250.0  262.5  275.0   
12                        1.0  260.000000         NaN  260.0  260.0  260.0   
20                        1.0   60.000000         NaN   60.0   60.0   60.0   
36                        1.0   36.000000         NaN   36.0   36.0   36.0   
50                        1.0  116.000000         NaN  116.0  116.0  116.0   
293                       1.0  232.000000         NaN  232.0  232.0  232.0   
1200                      2.0   34.000000    1.414214   33.0   33.5   34.0   
2004                      1.0   73.000000         NaN   73.0   73.0   73.0   
4000                      1.0   64.000000         NaN   64.0   64.0   64.0   
20171                     1.0   29.000000         NaN   29.0   29.0   29.0   
20172                     1.0   52.000000         NaN   52.0   52.0   52.0   
20173                     2.0   91.000000   35.355339   66.0   78.5   91.0   
20174                     1.0  267.000000         NaN  267.0  267.0  267.0   
28612                     1.0   47.000000         NaN   47.0   47.0   47.0   

                          
              75%    max  
Ambientes                 
0          225.00  996.0  
1           43.00  900.0  
2           59.00  894.0  
3           93.00  792.0  
4          160.00  860.0  
5          265.75  873.0  
6          340.00  751.0  
7          425.00  850.0  
8          396.50  847.0  
9          282.50  400.0  
10         840.00  840.0  
11         287.50  300.0  
12         260.00  260.0  
20          60.00   60.0  
36          36.00   36.0  
50         116.00  116.0  
293        232.00  232.0  
1200        34.50   35.0  
2004        73.00   73.0  
4000        64.00   64.0  
20171       29.00   29.0  
20172       52.00   52.0  
20173      103.50  116.0  
20174      267.00  267.0  
28612       47.00   47.0

In [67]:
# A reemplazar los 0 y los valores feos con una estimación. Usamos el 75% de los casos
for i, row in data_aux.iterrows():
    if row['Ambientes'] >= 10 or row['Ambientes'] <= 0:
        if row['surface_total_in_m2'] <= 42.00:
            data_aux.at[i, 'Ambientes'] = 1
        elif row['surface_total_in_m2'] <= 58.00:
            data_aux.at[i, 'Ambientes'] = 2
        elif row['surface_total_in_m2'] <= 92.00:
            data_aux.at[i, 'Ambientes'] = 3
        elif row['surface_total_in_m2'] <= 158.00:
            data_aux.at[i, 'Ambientes'] = 4
        elif row['surface_total_in_m2'] <= 267.00:
            data_aux.at[i, 'Ambientes'] = 5
        elif row['surface_total_in_m2'] <= 340.00:
            data_aux.at[i, 'Ambientes'] = 6
        elif row['surface_total_in_m2'] <= 430.00:
            data_aux.at[i, 'Ambientes'] = 7
        elif row['surface_total_in_m2'] > 430.00: 
            data_aux.at[i, 'Ambientes'] = 8

In [68]:
data_aux.groupby(by = 'Ambientes').describe()

surface_total_in_m2                                               \
                        count        mean         std    min    25%    50%   
Ambientes                                                                    
1                      5960.0   40.716611   33.489961   21.0   32.0   36.0   
2                      9934.0   53.283169   27.378926   21.0   44.0   50.0   
3                      9619.0   79.750806   29.721581   25.0   64.0   75.0   
4                      6660.0  127.470721   49.672445   25.0  100.0  120.0   
5                      4906.0  211.951488   50.032488   45.0  180.0  206.0   
6                      1679.0  298.291245   45.884101   74.0  280.0  300.0   
7                       941.0  377.397450   60.708489  110.0  351.0  377.0   
8                       969.0  568.472652  140.192701  116.0  473.0  528.0   
9                         7.0  233.857143   95.065641  132.0  180.0  180.0   

                         
             75%    max  
Ambientes                
1           41.0  900.0  
2           56.0  894.0  
3           86.0  792.0  
4          145.0  860.0  
5          239.0  873.0  
6          320.0  751.0  
7          400.0  850.0  
8          630.0  996.0  
9          282.5  400.0

In [69]:
# Ahora lo hago en el data_final

for i, row in data_final.iterrows():
    if row['Ambientes'] >= 10 or row['Ambientes'] <= 0:
        if row['surface_total_in_m2'] <= 42.00:
            data_final.at[i, 'Ambientes'] = 1
        elif row['surface_total_in_m2'] <= 58.00:
            data_final.at[i, 'Ambientes'] = 2
        elif row['surface_total_in_m2'] <= 92.00:
            data_final.at[i, 'Ambientes'] = 3
        elif row['surface_total_in_m2'] <= 158.00:
            data_final.at[i, 'Ambientes'] = 4
        elif row['surface_total_in_m2'] <= 267.00:
            data_final.at[i, 'Ambientes'] = 5
        elif row['surface_total_in_m2'] <= 340.00:
            data_final.at[i, 'Ambientes'] = 6
        elif row['surface_total_in_m2'] <= 430.00:
            data_final.at[i, 'Ambientes'] = 7
        elif row['surface_total_in_m2'] > 430.00: 
            data_final.at[i, 'Ambientes'] = 8

In [70]:
data_final.describe()

,ID,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,Ambientes,pileta,amenities,gimnasio,laundry,lavadero,balcon,terraza,sum,solarium,parrilla,a estrenar,subte,cochera
count,40675.000000,24637.000000,24637.000000,4.067500e+04,4.067500e+04,4.067500e+04,40675.000000,40675.000000,40675.000000,40675.000000,2354.000000,15511.000000,8.874000e+03,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000,40675.000000
mean,60739.269650,-34.839618,-58.642585,3.759226e+05,4.195848e+06,2.377992e+05,118.902200,107.104118,2138.306522,5205.258114,4.156330,2.765263,5.817462e+03,3.204204,0.260381,0.093817,0.080369,0.064831,0.351076,0.416374,0.238623,0.128728,0.084229,0.322557,0.118795,0.086392,0.405507
std,32899.987814,1.374966,1.343344,6.473110e+05,4.248888e+06,2.408052e+05,113.846588,268.364516,813.810879,11601.391773,4.911736,1.316765,1.513915e+05,1.666833,0.438848,0.291577,0.271867,0.246230,0.477312,0.492963,0.426247,0.334902,0.277734,0.467460,0.323551,0.280946,0.490996
min,0.000000,-54.806598,-71.663515,2.500000e+04,4.411125e+05,2.500000e+04,21.000000,1.000000,1000.158833,1.510204,1.000000,1.000000,1.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33212.000000,-34.632408,-58.552965,1.100000e+05,1.711516e+06,9.700000e+04,49.000000,44.000000,1523.809524,1775.510204,2.000000,2.000000,1.100000e+03,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,61393.000000,-34.597511,-58.448754,1.800000e+05,2.770186e+06,1.570000e+05,75.000000,66.000000,1984.615385,2346.153846,3.000000,3.000000,2.000000e+03,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,88319.000000,-34.551587,-58.395752,3.600000e+05,5.099260e+06,2.890000e+05,149.000000,128.000000,2569.382792,3193.159736,6.000000,3.500000,4.000000e+03,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,121219.000000,-24.181316,-54.584900,3.000000e+07,6.352020e+07,3.600000e+06,996.000000,49000.000000,5494.505495,650000.000000,133.000000,22.000000,1.000150e+07,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
#Crear y guardar la info en un nuevo csv
data_final.to_csv("./DATA/properattilimpio.csv")